In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
import seaborn as sns
import os
from sklearn import metrics
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
QuoraKaggle.pem
data_dump.zip
ec2-user@ec2-52-91-73-35.compute-1.amazonaws.com
sample_submission.csv
semantic_pos_features.csv
stanfordData_train.nlp
stanfordData_train_ner.nlp
test.csv
test_collins_duffy
test_dean
test_preprocessed
test_raw
train.csv
train_collins_duffy
train_dean



## Extract, Transform, Load

In [3]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")
df_submit = read_data("input/test.csv")

# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Load the Collins Duffy Training Features 

In [4]:
df_features = None
flip = True
base = 'input/train_collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': filename.replace(".csv","")})
        if flip:
            df_features = tmpFrame
            flip = False
        else:
            df_features = df_features.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-2_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-2_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e0_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e0_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_2e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_2e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_5e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_5e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_col

## Load the Collins Duffy Test Features

In [5]:
df_features_test = None
flip = True
base = 'input/test_collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': filename.replace(".csv","")})
        if flip:
            df_features_test = tmpFrame
            flip = False
        else:
            df_features_test = df_features_test.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-2_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-2_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e0_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e0_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_2e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_2e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_5e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_5e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/

In [6]:
TREEKERNEL_COL_LABELS = list(df_features.columns)
TREEKERNEL_COL_LABELS.remove('id')

## Load in Dean's Semantic Features

In [7]:
df_semantic = pd.read_csv('input/train_dean/dean_train_features.csv')
df_semantic.drop('is_duplicate',inplace=True,axis=1)

In [8]:
SEMANTIC_COL_LABELS = list(df_semantic.columns)
SEMANTIC_COL_LABELS.remove('id')

## Training the Model

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Read in the pre-built features for the training set
df_train = df_train.merge(df_features,how='inner',on='id')
df_train = df_train.merge(df_semantic,how='inner',on='id')

# Methodology - fill with zeroes Any row that crashed the Stanford CoreNLP parser must have been empty string
df_train.fillna(value=0, inplace=True)

## Resample the Training Set

In [10]:
target = 0.17
P = df_train[df_train['is_duplicate']==1].shape[0]
N = df_train[df_train['is_duplicate']==0].shape[0]
frac = ((P/target - P)/N)-1
n_add = round(frac * N)
## Rebalance the training set to match the test set percentage
neg_boostrap_sample = df_train[df_train["is_duplicate"] == 0].sample(n = n_add, replace = True)
rebalanced_df = pd.concat((neg_boostrap_sample, df_train))

## Extract the Features

In [11]:
select_cols = list(rebalanced_df.columns[6:])
df_y = rebalanced_df['is_duplicate']
df_X = rebalanced_df[select_cols].copy()

df_X.fillna(value=0, inplace=True)

In [12]:
# Rescale the training data
df_X[list(df_X.columns)] = MinMaxScaler().fit_transform(df_X[list(df_X.columns)])


## XGBoost

In [13]:
import xgboost as xgb

'''
alg: XGBClassifier() or XGBRegressor()
dtrain: the entire training dataframe
predictors: list of feature columns in dtrain
target: column we want to classify
'''

def modelfit(alg, dtrain, predictors, target, useTrainCV=True, cv_folds=5, early_stopping_rounds=4):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=target)
        cvresult = xgb.cv(xgb_param, 
                          xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'],
                          nfold=cv_folds,
                          metrics='logloss', 
                          early_stopping_rounds=early_stopping_rounds, 
                          verbose_eval=True)
        
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], target, eval_metric='logloss')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(target, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(target, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

/Users/Matthew/miniconda3/envs/bleedingEdge/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
#Choose all predictors except target & IDcols
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

modelfit(xgb1, df_X, SEMANTIC_COL_LABELS + TREEKERNEL_COL_LABELS, df_y.values)

[0]	train-logloss:0.643324+5.10788e-05	test-logloss:0.643337+3.99169e-05
[1]	train-logloss:0.602669+9.91706e-05	test-logloss:0.602713+4.48402e-05
[2]	train-logloss:0.568873+0.000104182	test-logloss:0.568947+5.45586e-05
[3]	train-logloss:0.540523+0.000128505	test-logloss:0.540609+7.00914e-05
[4]	train-logloss:0.516571+9.79767e-05	test-logloss:0.516676+8.10901e-05
[5]	train-logloss:0.496297+0.000129613	test-logloss:0.496416+4.98694e-05
[6]	train-logloss:0.478895+8.54602e-05	test-logloss:0.479016+0.000103924
[7]	train-logloss:0.463972+0.000104724	test-logloss:0.464109+0.000140354
[8]	train-logloss:0.451093+0.000118544	test-logloss:0.451249+0.000179746
[9]	train-logloss:0.439954+0.00012113	test-logloss:0.44012+0.000212608
[10]	train-logloss:0.430295+0.000137647	test-logloss:0.430469+0.000241096
[11]	train-logloss:0.421904+0.000105428	test-logloss:0.422103+0.000250143
[12]	train-logloss:0.414613+0.00011111	test-logloss:0.41483+0.000259504
[13]	train-logloss:0.408243+9.52617e-05	test-logloss

## Load in test set features

In [15]:
df_semantic_test = pd.read_csv('input/test_dean/dean_test_features.csv')

In [16]:
# Read in the pre-built features for the test set
df_submit = df_submit.merge(df_features_test,how='left',left_on='test_id',right_on='id')
# df_train = df_train.merge(df_semantic,how='inner',on='id')
df_submit.drop(['id'],inplace=True,axis=1)
df_submit = df_submit.merge(df_semantic_test,how='left',left_on='test_id',right_on='id')
df_submit.drop(['id'],inplace=True,axis=1)

# Methodology - fill with zeroes Any row that crashed the Stanford CoreNLP parser must have been empty string
df_submit.fillna(value=0, inplace=True)

In [17]:
# df_submit_features = df_submit[df_submit.columns[3:]].values

# d_test = xgb.DMatrix(df_submit_features)
# p_test = bst.predict(d_test)

# df_submit['is_duplicate'] = p_test

In [18]:
y_pred = xgb1.predict_proba(df_submit[SEMANTIC_COL_LABELS + TREEKERNEL_COL_LABELS])
# evaluate predictions
df_submit['is_duplicate'] = y_pred[:,1]

In [19]:
# The final submission set
df_final_submit = df_submit[['test_id','is_duplicate']]

In [ ]:
# plt.hist(df_submit['is_duplicate'], 50, normed=1, facecolor='green', alpha=0.75)

In [20]:
df_final_submit.to_csv("submissions/submission_xgb2_cv.csv",index=False)